<a href="https://colab.research.google.com/github/megha-goel/Data_Analsis_and_Visualization/blob/master/Finding_threshold_megha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import io
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from google.colab import files

Using TensorFlow backend.


LOADING DATA:

In [0]:
# UPLOAD FILE: METHOD 1 - FROM GITHUB
# Instructions: 1) open: https://github.com/megha-goel/protein_organism
#               2) select dataset.txt
#               3) click view raw
#               4) copy URL of the page that opens and store it in the variable url
url = 'https://raw.githubusercontent.com/megha-goel/protein_organism/master/dataset.txt?token=AILOHMVPKME6HCL6SNE2SH25L373E'
dataset = np.loadtxt(url, delimiter=',')

# NOTE: works for dataset.txt, takes a long time for label.txt
#       (make sure two raw files are not open at the same time in browser, open one, load and close, then open the next)
#       dataset_arr = genfromtxt(dataset, delimiter=',')   : might need it if it doesnt behave like an array

In [0]:
url = 'https://raw.githubusercontent.com/megha-goel/protein_organism/master/label.txt?token=AILOHMQTFBWZE32Z5Y7DGN25L4AT6'
label = np.loadtxt(url, delimiter=',')

In [24]:
if (np.shape(dataset)[0] != np.shape(label)[0]):
  print("number of dataset entries is not equal to the number of labels! CHECK!")
else:
  print("EVERYTHING LOOKS GOOD! PROCEED..")

EVERYTHING LOOKS GOOD! PROCEED..


In [0]:
# UPLOAD FILE: METHOD 2 - FROM LOCAL DIRECTORY
# Instructions: 1) run this cell (might give an error, in which case just run it again..)
#               2) 'Choose Files' option appears
#               3) from the local directory, select the dataset file
#               4) then run the next cell to load that data into dataset variable
#               5) do the same for label.txt
uploaded = files.upload()

In [0]:
dataset = loadtxt(io.BytesIO(uploaded['dataset.txt']), delimiter = ',')

In [0]:
uploaded = files.upload()

Saving dataset.txt to dataset.txt


In [0]:
labels = loadtxt(io.BytesIO(uploaded['label.txt']))

DATA LOADED! NOW DIVIDING DATA INTO TRAIN AND TEST SETS:

In [0]:
num_total = np.shape(label)[0]
num_train = int(0.7*num_total)
num_test = num_total - num_train
train_x = dataset[0:num_train, :]
train_y = label[0:num_train]
test_x = dataset[num_train:, :]
test_y = label[num_train:]

In [33]:
# check to see if all good

if (len(test_x) != len(test_y)):
  print("number of TEST datapoints NOT EQUAL to number of test labels! CHECK!")
else:
  print("ALL GOOD! PROCEED..")

ALL GOOD! PROCEED..


In [0]:
# better way to split test and train once the uploaded files are corrected and have same dimensions
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

BUILDING, TRAINING AND TESTING MODELS:

In [35]:
# A simple neural network with dense layers
model_NN = Sequential()
model_NN.add(Dense(100, input_dim=(3), activation='relu'))
model_NN.add(Dense(200, activation='relu'))
model_NN.add(Dense(50, activation='relu'))
model_NN.add(Dense(1, activation='sigmoid'))
model_NN.summary()
# compile the keras model
model_NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0822 20:59:57.598014 140341224896384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0822 20:59:57.621901 140341224896384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0822 20:59:57.630918 140341224896384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 100)               400       
_________________________________________________________________
dense_6 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 30,701
Trainable params: 30,701
Non-trainable params: 0
_________________________________________________________________


In [37]:
# train model and see test accuracies through validation at each epoch
model_NN.fit(train_x, train_y, epochs=2, batch_size=1000, validation_data = (test_x, test_y))

Train on 228400 samples, validate on 97886 samples
Epoch 1/2
228400/228400 [==============================] - 3s 12us/step - loss: 0.1268 - acc: 0.9590 - val_loss: 0.1107 - val_acc: 0.9618
Epoch 2/2
228400/228400 [==============================] - 3s 12us/step - loss: 0.1225 - acc: 0.9592 - val_loss: 0.1173 - val_acc: 0.9563


In [0]:
predprob_y = model_NN.predict_proba(test_x)
pred_y = model_NN.predict(train_x)
#saving output
np.savetxt('predicted_classes.txt',pred_y, delimiter = ',')
np.savetxt('probabilities_predclas.txt',predprob_y, delimiter = ',')

In [0]:
files.download('predicted_classes.txt')

In [0]:
files.download('probabilities_predclas.txt')

K-MEANS CLUSTERING:

In [40]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=2) # You want cluster the passenger records into 2: Survived or Not survived
kmeans_model.fit(train_x)
# KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
#    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
#    random_state=None, tol=0.0001, verbose=0)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [41]:
#Accuracy 
correct = 0
for i in range(len(test_x)):
    predict_me = np.array(test_x[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans_model.predict(predict_me)
    if prediction[0] == test_y[i]:
        correct += 1

print(correct/len(test_x))


0.8744866477330773


SVM:

In [2]:
# An SVM classifier using scikit
#takes too long.. probably some bug

from sklearn.svm import SVC
model_svm = SVC(gamma='auto')
model_svm.fit(train_x, train_y)


NameError: ignored

In [0]:
pred_y = model_svm.predict(test_x)
print(confusion_matrix(test_y,pred_y))
print(classification_report(test_y,pred_y))